<a href="https://colab.research.google.com/github/amelierueeck/ULM-25-authorship-profiling/blob/main/src/BERT_probing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code from Polina

In [ ]:
# !pip install spacy ftfy==4.4.3
# !python -m spacy download en

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import sys
import os

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device('cpu')

[more here - in Polina's notebook: getting the model and the tokenizer, getting sentence representations, getting data]

**Simple linear classifier**

In [ ]:
class Classifier(torch.nn.Module):

  def __init__(self, input_dim, output_dim):
    super(Classifier, self).__init__()
    self.linear = totch.nn.Linear(input_dim, output_dim)

  def forward(self, input):
    return self.linear(input)

def build_classifier(emb_dim, num_labels, device):
  classifier = Classifier(emb_dim, num_lables).to(device)
  criterion = torch.nn.CrossEntropyLoss().to(device)
  optimizer = torch.optim.Adam(classifier.paramters())

  return classifier, criterion, optimizer